# Demo OCR

In [1]:
import ollama
import PIL.Image
from  pdf2image import pdf2image
import io
from pathlib import Path
import base64
import os

In [4]:
# Convert pdf to image
path = r"C:\Users\FADELCO\Downloads\Immatriculation DGI NIF.pdf"
images = pdf2image.convert_from_path(pdf_path=path,
                                     thread_count=2
                                     )

In [ ]:
for i,img in enumerate(images):
    s = Path("../data") / (f"{i}-" +  Path(path).with_suffix(".jpg").name)
    img.save(s)

In [48]:
prompt = "Extract the text from the image. Reply directly"

In [5]:
image_dir = "../data"

In [39]:
image_paths = list(Path(image_dir).glob("*.jpg"))
image_paths

[WindowsPath('../data/0-Immatriculation DGI NIF.jpg'),
 WindowsPath('../data/1-Immatriculation DGI NIF.jpg')]

### Ollama

In [34]:
response = ollama.generate(
    model="gemma3:4b",  # Choose your desired model
    prompt=prompt,
    images=[image_paths[0]],
    options={"temperature": 0.1}
)

In [35]:
print(response["response"])


Here's the extracted text from the image:

**Certificat d'Immatriculation**

**Le contribuable :**

Nom, Prénoms ou Raison Sociale : SEYDOU FADEL MAMAR

Nom Commercial : DELCAUX CONSULTING

Forme Juridique : ENTREPRISE INDIVIDUELE

Numéro du Registre de Commerce : NE/NM/2021/A10/00248 DU 21/01/2021

Activité : COMMERCE GENERAL / PRESTATION DE SERVICES / FORMATION

Adresse : B.P. :
NIAMEY

Tél : 87366554

Section : CUNI

Ville : NIAMEY

Tél : 207221661

Pour Le Directeur Général des Impôts P.O.

134517 / R

En conséquence, l'intéressé est tenu de faire figurer sur tous les documents professionnels (correspondances, factures, déclarations...) le dit numéro sous peine de sanctions prévues par les textes en vigueur.


In [33]:
response

GenerateResponse(model='gemma3:4b', created_at='2025-06-05T14:39:23.4448365Z', done=True, done_reason='stop', total_duration=130797389400, load_duration=64352300, prompt_eval_count=538, prompt_eval_duration=86627562300, eval_count=500, eval_duration=44081539700, response="Here's the extracted text from the image:\n\n**I. CONDITIONS D'UTILISATION**\n\nLe NIF est un numéro unique, exclusif et invariable. Il ne peut servir qu'à celui à qui il est attribué. Le contribuable doit mentionner son numéro d'identification fiscale.\n\n- sur les déclarations fiscales et douanières;\n- sur les factures délivrées à ses clients;\n- sur les documents professionnels remis à des tiers;\n- lors de l'ouverture de comptes, pour les professionnels, auprès des établissements bancaires ou financiers.\n\n**II. SUSPENSION, REACTIVATION DE RADIATION**\n\n**SUSPENSION**\n\nLe NIF peut faire l'objet de suspension:\n\n- à initiative de l'administration fiscale en cas de non respect notoire des obligations fiscales 

### Gemini

In [78]:
from llama_index.multi_modal_llms.gemini import GeminiMultiModal
from llama_index.core import SimpleDirectoryReader
from llama_index.core.llms import ChatMessage, ImageBlock,TextBlock
import json
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.core.bridge.pydantic import BaseModel
from typing import Sequence
from tqdm import tqdm
from dotenv import load_dotenv


In [ ]:
load_dotenv(dotenv_path="../.env")

True

In [6]:
image_documents = SimpleDirectoryReader(image_dir).load_data()

In [75]:
def perform_ocr(prompt:str,image_paths:Sequence, llm):

    extracted_txt = []
    
    for path in tqdm(image_paths,desc="Running OCR..."): 

        msg = ChatMessage(
            role="user",
            blocks=[
                ImageBlock(path=path),
                TextBlock(text=prompt),
            ],
        )

        resp = llm.chat([msg])

        text = resp.message.blocks[0].text

        extracted_txt.append(text)

    return extracted_txt

In [60]:
class OCR(BaseModel):
    """A parsed image's content using Optical character recognition."""

    context: str

In [73]:
model = "gemini-2.5-flash-preview-04-17"

llm = GoogleGenAI(
    model=model,
    temperature=0.1
) #.as_structured_llm(OCR)

# llm = GeminiMultiModal(model="models/" + model,temperature=0.1)

In [74]:
print(model,"\n\n")

extracted_text = perform_ocr(prompt=prompt,image_paths=image_paths,llm=llm)

print(extracted_text)

gemini-2.5-flash-preview-04-17 


["République du Niger\nMinistère de l'Economie et des Finances\nDirection Générale des Impôts\nIGI\nLe 07/02/2025\n\nCertificat d'Immatriculation\n\nLe contribuable :\n\nNom, Prénoms ou Raison Sociale : SEYDOU FADEL MAMAR\n\nNom Commercial : DELCAUX CONSULTING\nForme Juridique : ENTREPRISE INDIVIDUELLE\nNuméro du Registre de Commerce : NE/NIM/01/2025/A10/00248 DU 21/01/2025\nActivité : COMMERCE GENERAL/PRESTATION DE SERVICES/FORMATION\n\nAdresse : B.P. : Quartier : SONUCI Ville : NIAMEY\nTél : 87366554 Rue :\nSecteur/Section : CUN I Ilot : Parcelle :\nMarché : Boutique N° :\n\nA été immatriculé sous le numéro d'identification fiscale (NIF) : 134517 / R\nà la suite de sa prise en compte dans les services de la Direction Générale des Impôts.\n\nEn conséquence, l'intéressé est tenu de faire figurer sur tous les documents professionnels (correspondances, factures, déclarations...) ledit numéro sous peine de sanctions prévues par les textes en vigueur.\n\nS

In [77]:
extracted_text[-1]

"Certificat d'immatriculation\n\nI. CONDITIONS D'UTILISATION\n\nLe NIF est un numéro unique, exclusif et invariable. Il ne peut servir qu'à celui à qui il est attribué. Le contribuable doit\nmentionner son numéro d'identification fiscale :\n- lors des démarches entreprises auprès des administrations;\n- sur ses déclarations fiscales et douanières;\n- sur les factures délivrées à ses clients;\n- sur les documents professionnels remis à des tiers;\n- lors de l'ouverture de comptes, par les professionnels, auprès des établissements bancaires ou financiers.\n\nII. SUSPENSION, REACTIVATION ET RADIATION\n\n•SUSPENSION\nLe NIF peut faire l'objet de suspension :\n- à l'initiative du contribuable qui en fait la demande motivée;\n- à l'initiative de l'administration fiscale en cas de non respect notoire des obligations fiscales par le contribuable\nLa suspension du NIF produit les mêmes effets que le défaut de NIF\n\n•REACTIVATION\nLa réactivation d'un NIF suspendu est subordonnée à la constitut

In [66]:
print(json.loads(extracted_text)["context"])

République du Niger Ministère de l'Economie et des Finances Direction Générale des Impôts IGI Le 07/02/2025 Certificat d'Immatriculation Le contribuable : Nom, Prénoms ou Raison Sociale : SEYDOU FADEL MAMAR Nom Commercial : DELCAUX CONSULTING Forme Juridique : ENTREPRISE INDIVIDUELLE Numéro du Registre de Commerce : NE/NIM/01/2025/A10/00248 DU 21/01/2025 Activité: COMMERCE GENERAL/PRESTATION DE SERVICES/FORMATION Adresse : B.P. : Quartier : SONUCI Ville : NIAMEY Tél : 87366554 Rue : Secteur/Section : CUNI llot : Parcelle : Marché : Boutique N° : A été immatriculé sous le numéro d'identification fiscale (NIF) : 134517 / R à la suite de sa prise en compte dans les services de la Direction Générale des Impôts. En conséquence, l'intéressé est tenu de faire figurer sur tous les documents professionnels (correspondances, factures, déclarations...) ledit numéro sous peine de sanctions prévues par les textes en vigueur. SERVICE FISCAL DE RATTACHEMENT Nom : CENTRE DES IMPOTS YANTALA Adresse : Y